# Chapter 4: 실제 데이터를 텐서로 표현해보기



## 이미지를 텐서로

### 특징

- 이미지는 픽셀 단위의 높이와 너비를 가지는 그리드에 나열된 여러 개의 스칼라 값으로 표현된다.
- 한 픽셀에 대해 하나의 스칼라 값을 가지는 흑백 이미지부터 여러 색을 표현하기 위해 여러 개의 스칼라 값을 가지기도 한다.
- 색상 외에, 카메라에서 얻은 심도 등 추가적인 피처가 포함되기도 한다.


In [1]:
import imageio
import numpy as np
import torch

In [3]:
img_arr = imageio.imread('data/p1ch4/image-dog/bobby.jpg')
img_arr.shape


/var/folders/4l/lb4d41pn56d4wnwsr9rxl20c0000gn/T/ipykernel_10244/3653726803.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread('data/p1ch4/image-dog/bobby.jpg')


(720, 1280, 3)

In [9]:
# 채널, 높이, 너비 순으로 차원을 나열해야 하므로, permute를 사용하여 차원을 변경한다.
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)
print(id(img.storage().data_ptr()), id(out.storage().data_ptr()))

4842263568 4842263568


`permute`와 `transpose`의 차이점은 `permute`는 모든 차원을 재배열할 수 있는 반면, `transpose`는 두 차원만 교환한다는 점이다.

In [5]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [7]:
### 배치 입력
import os

data_dir = 'data/p1ch4/image-cats'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.png']

for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3] # 투명도, 알파 채널 등이 포함된 이미지라도 RGB 채널, 높이, 너비만을 사용한다
    batch[i] = img_t

/var/folders/4l/lb4d41pn56d4wnwsr9rxl20c0000gn/T/ipykernel_10244/1534984906.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread(os.path.join(data_dir, filename))


In [10]:
### 데이터 정규화
# 첫 번째 방법. 단순히 최댓값으로 나누기
batch = batch.float()
batch /= 255.0  # 0-255 범위의 값을 0-1 범위로 정규화

In [13]:
# 두 번째 방법. 각 채널별로 평균과 표준편차를 이용한 정규화
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.mean(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std  # 각 채널별로 정규화


※ `batch[:, c]` 해석

- `:` 는 해당 차원 전체를 의미
- `c` 는 두 번째 차원(채널)에 해당하는 인덱스

## 3차원 이미지

CT 스캔, MRI 등은 3차원 이미지로 표현된다. 이들은 2차원 이미지의 스택으로 볼 수 있다.

채널 차원 뒤에 깊이 차원이 추가된다.

In [4]:
import imageio

dir_path = 'data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083'
vol_arr = imageio.volread(dir_path, 'DICOM')
vol_arr.shape

Reading DICOM (examining files): 99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 99/99  (100.0%)


(99, 512, 512)

In [7]:
# 채널 추가
vol = torch.from_numpy(vol_arr)
vol = torch.unsqueeze(vol, 0) # 맨 앞에 차원 추가

vol.shape

torch.Size([1, 99, 512, 512])

## 테이블 데이터

스프레드시트 또는 CSV 파일, 데이터베이스의 데이터에 대한 입력 처리

In [2]:
import csv

wine_path = 'data/p1ch4/tabular-wine/winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';', skiprows=1)

wineq_numpy[:5]

array([[7.000e+00, 2.700e-01, 3.600e-01, 2.070e+01, 4.500e-02, 4.500e+01,
        1.700e+02, 1.001e+00, 3.000e+00, 4.500e-01, 8.800e+00, 6.000e+00],
       [6.300e+00, 3.000e-01, 3.400e-01, 1.600e+00, 4.900e-02, 1.400e+01,
        1.320e+02, 9.940e-01, 3.300e+00, 4.900e-01, 9.500e+00, 6.000e+00],
       [8.100e+00, 2.800e-01, 4.000e-01, 6.900e+00, 5.000e-02, 3.000e+01,
        9.700e+01, 9.951e-01, 3.260e+00, 4.400e-01, 1.010e+01, 6.000e+00],
       [7.200e+00, 2.300e-01, 3.200e-01, 8.500e+00, 5.800e-02, 4.700e+01,
        1.860e+02, 9.956e-01, 3.190e+00, 4.000e-01, 9.900e+00, 6.000e+00],
       [7.200e+00, 2.300e-01, 3.200e-01, 8.500e+00, 5.800e-02, 4.700e+01,
        1.860e+02, 9.956e-01, 3.190e+00, 4.000e-01, 9.900e+00, 6.000e+00]],
      dtype=float32)

In [3]:
# wineq_numpy 의 각 열에 대해 통계 자료를 출력
for i in range(wineq_numpy.shape[1]):
    col = wineq_numpy[:, i]
    print(f'Column {i}: mean={col.mean():.2f}, std={col.std():.2f}, min={col.min():.2f}, max={col.max():.2f}')


Column 0: mean=6.85, std=0.84, min=3.80, max=14.20
Column 1: mean=0.28, std=0.10, min=0.08, max=1.10
Column 2: mean=0.33, std=0.12, min=0.00, max=1.66
Column 3: mean=6.39, std=5.07, min=0.60, max=65.80
Column 4: mean=0.05, std=0.02, min=0.01, max=0.35
Column 5: mean=35.31, std=17.01, min=2.00, max=289.00
Column 6: mean=138.36, std=42.49, min=9.00, max=440.00
Column 7: mean=0.99, std=0.00, min=0.99, max=1.04
Column 8: mean=3.19, std=0.15, min=2.72, max=3.82
Column 9: mean=0.49, std=0.11, min=0.22, max=1.08
Column 10: mean=10.51, std=1.23, min=8.00, max=14.20
Column 11: mean=5.88, std=0.89, min=3.00, max=9.00


In [4]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))

wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [5]:
wineq = torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [6]:
# 마지막이 타겟 레이블이므로 제외
data = wineq[:, :-1]

data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [7]:
# 타겟 텐서 전치
target = wineq[:, -1].long()  # 레이블은 정수형으로 변환

### 레이블을 처리하는 방식

1. 위의 예제처럼 레이블을 정수형으로 변환하여 사용
2. 원-핫 인코딩(One-hot encoding)으로 변환하여 사용

레이블을 정수 벡터로 처리하는 것은 값에 순서가 있다는 것이며, 위의 예제는 와인 점수를 예측하는 문제를 풀려고 하기 때문에 적절하다.
또, 값 사이의 거리도 의미가 있게 된다. 예를 들어, 와인 품질 점수 1과 3의 차이는 2와 4의 차이와 같다고 보는 것이다. 실제 품질이 점수 관계와 동일하다면 이는 적절하다.

그러나 만약 타겟이 포도 품종처럼 서로 간에 완전히 이산적인 경우라면 값 사이의 순서나 거리 개념이 없는 원 핫 인코딩이 더 적절할 수 있다.

In [9]:
target.unsqueeze(dim=1)

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

In [11]:
# 원핫 인코딩 사용 방법
target_onehot = torch.zeros(target.shape[0], 10) # shape: (batch_size, num_classes)

# scatter_ -> dim: _int, index: Tensor, src: Tensor
target_onehot.scatter_(dim=1, index=target.unsqueeze(dim=1), src=1.0)
# 두 번째 차원(클래스 축)에 대해 인덱스에 해당하는 위치에 1.0을 채운다.

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [12]:
# 입력 텐서 정규화
mean = torch.mean(data, dim=0)
var = torch.var(data, dim=0)

data = (data - mean) / torch.sqrt(var + 1e-8)  # 분모에 작은 값 추가하여 0으로 나누는 것을 방지